In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import re
import pandas as pd
from pathlib import Path
from terra_mystica_models.data import make_dataset

In [3]:
json_dir = Path(".").resolve().parent / "data" / "raw"

In [4]:
eg_json = json_dir / "2014-01.json"
with open(eg_json) as f:
    data = json.load(f)

Regarding your bare minimum only use games played n the base original map 126fe960806d587c78546b30f1a90853b1ada468 (edited)

You'll want to avoid games that have fire-and-ice-factions fire-and-ice-factions/ice fire-and-ice-factions/variable_v5 fire-and-ice-factions/volcano fire-and-ice-final-scoring

these are the expansion factions + expansion scoring

they are played online and are interesting, but for now if we are staring simple, then ignore them

I have no idea what nofaction is, just ignore any games with it.

fire-and-ice-factions/ice are icemaidens and yetis

fire-and-ice-factions/volcano are dragonlords and acolytes

fire-and-ice-factions/variable_v5 are shapeshifters and riverwalkers

SCORE 9 is an expansion scoring tile that has been added to the "base" game because it's really good. See here: https://tuesdaydd.slack.com/archives/D0EFM6XST/p1582510066003500

Becasue online games are timed (otherwise they'd never finish), players can actively be dropped from games. It happens lots in casual games, less so in ranked games, virtually never in high ranked games.

As such excluding any games that have dropped players is actually a good way of "curating" results from better players. (edited)

Ok, I think I have most of what I need. I still need to figure out play order though so I can build models depending on whether you have first, second choice etc.

In [5]:
eg_game = data[10]

In [6]:
eg_game["factions"]

[{'player': 'Duilen', 'faction': 'witches'},
 {'player': 'Lewdog', 'faction': 'mermaids'}]

In [7]:
eg_game["events"]["faction"]["witches"]["order:1"]

{'turn': {'5': {'1': 1}, '3': {'1': 1}}, 'round': {'all': 2, '5': 1, '3': 1}}

In [8]:
eg_game["events"]["faction"]["witches"]["order:2"]

{'turn': {'6': {'1': 1}, '1': {'1': 1}, '4': {'1': 1}, '2': {'1': 1}},
 'round': {'6': 1, 'all': 4, '1': 1, '4': 1, '2': 1}}

This is turn order for in the game, but there's nothing for turn 0 where I assume faction selection and initial placement happens

From discussion with Kyle order of play in round 1 should be equal to order of faction selection, so I'll go with that.

# Assemble the elements

In [9]:
tmg = make_dataset.TerraMysticaGame(eg_game)

In [10]:
tmg.game_name

'2p4'

In [11]:
tmg.game_date

datetime.datetime(2014, 1, 16, 22, 28, 53)

In [12]:
tmg.base_map

'126fe960806d587c78546b30f1a90853b1ada468'

In [13]:
tmg.players

{'witches': 'Duilen', 'mermaids': 'Lewdog'}

In [14]:
tmg.factions

['witches', 'mermaids']

In [15]:
tmg.faction_selection_order

{'mermaids': 1, 'witches': 2}

In [16]:
tmg.scoring_tile_order

{'SCORE5': 5, 'SCORE4': 2, 'SCORE7': 6, 'SCORE3': 4, 'SCORE8': 3, 'SCORE2': 1}

In [17]:
tmg.bonus_tiles

['BON9', 'BON4', 'BON3', 'BON5', 'BON6']

In [18]:
tmg.victory_points

{'witches': 117, 'mermaids': 114}

In [19]:
tmg._check_player_count()

True

In [20]:
tmg._check_bonus_tile_count()

True

In [21]:
tmg._is_played_on_original_map()

True

Still to add:
More validation. As described above there are certain game options and factions I want to restrict from my analysis so I need to have checkers for that. I also want to know if any players dropped from a game so I can potentially filter on that

After that I can actually use this class to turn all these games into some datasets